# ML feature engineering explained

"At the end of the day, some machine learning projects succeed and some fail.  What makes the difference?  Easily the most important factor is the features used."
-- Prof. Pedro Domingos


## Summary

This notebook is intended to walk through the process of feature engineering using some common tools and methodologies.  This lab and its associated material is not intended to be exhaustive but an introduction to the concept, challenges, and objectives of feature engineering.  

There are numerous writings about feature engineering and readers are encouraged to seek them out beyond this notebook.

This notebook assumes you have some proficiency with the Python programming language and will introduce some of the features of the Pandas Python library as well as the scikit-learn library.  A general level of comfort with Jupyter notebooks is also assumed.

You will start by downloading a raw data set from the Lending Club.  Once downloaded you will explore the data using various features of the Pandas library and its DataFrame class.  After exploring the data you will begin to transform the data using the same DataFrame class and its manipulation functions.  After the data has been modified to be suitable for the logistic regression classification algorithm, you will use the scikit-learn library to quickly train and test the performance of the data using logistic regression.

The goal is to predict the [loan grade](https://www.investopedia.com/terms/l/loan-grading.asp) of a loan based on characeteristics such as loan amount, income, and time of year. 

This notebook will use a data set made available by the Lending Club.  The data set and data catalog can be found at:

https://www.lendingclub.com/info/download-data.action

For this notebook use the Q1 2016 CSV data set. 

To predict 

### Lab steps
 1. Download the data
 1. Explore the data
     1. Examine the raw data, its shape, and sample values
     1. View histograms and correlation matrices of the features
 1. Feature engineering
     1. Remove redundant or incomplete data
     1. Format the data and set its data type
     1. Decompose dates, categories, and numerical quantities
     1. Normalize and standardize the features
 1. Test the features using logistic regression
 
 **Extra credit**
 1. Use principal component analysis to reduce the number of features
 1. Re-evaluate with the new training data set to see the effect of PCA on logistic regression performance
 

## Further reading:

https://medium.com/ai%C2%B3-theory-practice-business/top-6-errors-novice-machine-learning-engineers-make-e82273d394db
https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/
https://machinelearningmastery.com/quick-and-dirty-data-analysis-with-pandas/
https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b
https://machinelearningmastery.com/visualize-machine-learning-data-python-pandas/


---
# DOWNLOAD THE SOURCE DATA

The data for this tutorial is the Q1 2016 CSV data set available on the [Lending Club website](https://www.lendingclub.com/info/download-data.action).  The commands below will download the data set and unzip it to the local system.

In [ ]:
!wget https://resources.lendingclub.com/LoanStats_2016Q1.csv.zip
!unzip LoanStats_2016Q1.csv.zip && rm LoanStats_2016Q1.csv.zip
!mkdir data && mv LoanStats_2016Q1.csv data/loan_stats_2016q1.csv

---

# EXPLORE THE DATA
---

## Take a peek at the raw data

Use a shell command to get the first 5 lines of data from the CSV

In [ ]:
!head -5 data/loan_stats_2016q1.csv

## Read the data into memory using Pandas

[Pandas](https://pandas.pydata.org/) is an open source library that provides data structures and tools for data analysis for the Python programming language.

In the cell below use the [read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) function of pandas to convert the CSV at `data/loan_status_2016q1.csv` to a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [ ]:
import pandas as pd

# read the CSV into a pandas DataFrame
DATA_FILE_NAME = 'data/loan_stats_2016q1.csv'
# 'df' is a common abbreviation for data frame and is used in many code samples to represent the data object
df = pd.read_csv (
    
    ...
    
    )

## Using summary functions to further explore the data

Jupyter notebooks will automatically concatenate fields in order to not try to display too much at once.  Using the `set_option` function of Pandas we will configure the notebook to display up to 145 columns of our DataFrame.

In [ ]:
# the data is 145 columns wide, we will need to allow for all the columns when displaying the data
pd.set_option ('display.max_columns', 145)

In the cell below use the [info](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html) function to display information such as record count, number of columns, data types, and how much memory is used by the DataFrame that now houses the CSV data.

In [ ]:
# display a summary of the data frame shape


We see that the DataFrame contains 133,889 rows of data with 145 columns.  It has two data types, float and object, and is using 148 MB of RAM.

Next obtain 5 example rows of data using the [sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) function.

In [ ]:
# print a tabulated list of 5 random rows of data


On the far left is the row number for every sample record.  You will also see some fields have no value or are shown as 'NaN'.  This indicates missing data which may or may not be acceptable given your use case.

Next use the [describe](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) function to list common calculations such as average, standard deviation, min, and max values for all numerical fields in the data set.

In [ ]:
# list common statistics per column for the data frame


The describe function will provide statistics for every column of a float data type.  This will include how many values are in the column (vs NaN values), the average, standard deviation, and different percentiles.

Next list the data types for each column using the `dtypes` attribute of the DataFrame object.

In [ ]:
# retrieve a list of the data types for each column
pd.set_option ('display.max_rows', 145)


Now that we can see all the various fields and their data type lets get a sample of the fields that have the ``object`` data type.  Perhaps we can convert these to a data type that is better suited for machine learning.  Use the [select_dtypes](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html) function to show a sample of 5 values for each field which is of type `object`.

In [ ]:
# show those columns that have an 'object' data type


Looking at the values from the sample above many of these are best suited as a ``category`` data type, a field with a finite set of potential values.  Others are percentages that can be converted to floating point numbers or dates.

### Data value distribution

Histograms can be used to get a feel for the distribution of values within any field in a data set.  Using the ``hist`` function of a Pandas DataFrame you can easily see how the values are distributed.  Look for those fields that have a Gaussian, exponential, or skewed distribution.  This will help you identify outliers in your data set.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

df.hist (figsize=(25,25))
plt.show ()

### Correlation matrix

Using a correlation matrix you can quickly get a feel for where correlation may reside in your data set.  Those fields that are yellow demonstrate a high correlation and those that are blue demonstrate a high inverse correlation.  

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

df_float = df.select_dtypes(include='float64')
col_names = df_float.columns.tolist()

correlations = df_float.corr()

# plot correlation matrix
fig = plt.figure(figsize=(25,25))
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(col_names),1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(col_names)
ax.set_yticklabels(col_names)
plt.xticks(rotation=90)
plt.show()

The fields that are white indicate null values in the data set.

### Data exploration conclusion

You should now have a sense of the data that is in the data set, how many columns there are, how many rows, and have a sense of what data types are in the data set.  In addition you should have an appreciation for the quality of the data as it stands.  Many fields are empty or have ``object`` or string value types that we will need to convert for them to be useful for ML purposes.  Lets now dig into the data to determine which fields are useful, which can be removed, and which need to be modified to help describe our problem space and enable us to develop an effective ML model.

# FEATURE ENGINEERING

## Data cleansing

### Remove unnecessary or redundant columns

Looking at the ``sample`` displays above it appears that some of the columns may have duplicate data or data that is perhaps irrelevant to the problem we are trying to solve.  Lets explore these further and remove them if necessary.

Use the ``unique`` function for any column of the data to get a list of the potential values.

In [ ]:
df.purpose.unique ()

Using the same [unique](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) function list the values for the `title` column.

The `title` and `purpose` columns appear to be redundant.  Use the [drop](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html) function to delete the `title` column and the [upper](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.upper.html) function to capitalize the values in the `purpose` column.

We will not be trying to derive value from the job titles so for now, lets remove them from our feature set using the `drop` function.

### Remove columns that contain 10% or more NaN values

We want to ensure that of the data we have that it is of a certain quality.  For any columns that contain more than 10% of null or empty values remove the column as it doesn't contain enough data to be of value.

Lets start by using the [isna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.isna.html) and [sum](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sum.html) functions to list each column and count the NaN values in the column.

In [ ]:
print ("DataFrame before null removal")
df.info (verbose = False)

Now use the [dropna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) function to remove those columns that have more than 10% of NaN values.

In [ ]:
print ("DataFrame AFTER null removal")
df.info ()

Based on the before and after we can see that we have gone from 143 columns to 97 columns, more than 45 columns had more than 10% empty data!  

**Note**: 10% is a threshold set for the purposes of this lab, the threshold that is suitable for your data will be dependent upon the algorithm you are training with and the data you are using.

### Remove rows that contain ALL NaN values

We have now cleared columns with too much empty data.  Lets now use the same `dropna` function to remove any rows that contain nothing but NaN values

## Data formatting

Use the `sample` function again to retrieve another 5 example rows of data.  Look at the values and determine where unnecessary formatting can be removed such as '%' symbols or labels such as 'years'.

In [ ]:
# Remove unnecessary formatting from field values in prep for setting data types


### Clean the term column

Using the `unique` function to assess the `term` column - is there a unit of measure that can be stripped to convert the value into a numerical data type?  Using the [apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) function strip the label from the text and convert it to a number.

In [ ]:
# note that term contains 133887 values, but there are 133889 rows of data


In [ ]:
# convert the string value of term to an int


### Clean the int_rate column

The interest rate is also being stored as an object at the moment for the sake of its '%' sign, remove this and format the number as a float.

In [ ]:
# convert the interest rate field to a float value


### Clean the revol_util column

Like the interest rate the ``revol_util`` column also contains a '%' sign, remove it.

In [ ]:
# clean the revol_util column, it contains '%' signs


### Cleanup dates

The month is a cyclical value, after December comes January, so December (12) is closer to January (1) than perhaps October (10).  1 month between Dec and Jan vs 3 months between October and January.  We need to represent this cyclical nature using the geometrical functions sin and cos.

For more read http://blog.davidkaleko.com/feature-engineering-cyclical-features.html

In [ ]:
# convert a month / year date to an individual month and year column
from numpy import cos, sin, pi

def decompose_date (pd_df, column_name):
    date_column = pd.to_datetime(pd_df[column_name])
    month = date_column.dt.month
    pd_df[column_name +"_month_cos"] = cos ((month-1)*(2.0*pi/12)) # subtract 1 so that jan is month 0
    pd_df[column_name +"_month_sin"] = sin ((month-1)*(2.0*pi/12))
    pd_df[column_name +"_year"] = date_column.dt.year.astype (float)
    pd_df[column_name +"_absolute"] = pd.to_numeric(date_column.dt.strftime ('%s'), errors = 'coerce') # store the unix timestamp
    pd_df.drop(columns=[column_name], inplace = True)

In [ ]:
decompose_date (df, 'issue_d')
decompose_date (df, 'earliest_cr_line')
decompose_date (df, 'last_pymnt_d')
decompose_date (df, 'last_credit_pull_d')

In [ ]:
df.sample (5)

In [ ]:
df.dtypes

## Set Data types

We have refined the values in our data but based on the data types listing above you can see that there are still a few ``object`` data types.  We can go through each one of these types by hand and determine if we can further refine them to float or int values but for now lets set them all to categorical items.  A category is a value that has a finite set of values such as payment types ('cash', 'credit', 'check') or day of the week ('monday', 'tuesday', etc).  Convert all remaining ``object`` types to a ``category`` type.

When fed to an ML algorithm the ``category`` data types will be 'one hot' encoded to a numerical value.  In the example of payment types this will be a 3 digit vector where a value of 1 is set in place of the correct value and others are set to 0.  For example a payment type of 'credit' would be one hot encoded to an array such as [0, 1, 0].

### Set categorical columns

Use the [select_dtypes](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html) function to obtain an iterator over all features of type `object`.  Then use the [astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html) function to convert each to a type of `category`.

In [ ]:
# any column of type object that does not contain NaN, set its data type to categorical


In [ ]:
df.dtypes

### Data cleanse conclusion

You have now removed a majority of the NaN or empty values, formatted the data to be of a data type that an ML algorithm will understand and you converted values to meaningful float values where appropriate.  The next step is to use scaling and normalization to bring values within relative magnitude of one another.


## Scale the features

Looking at a sample of the data above you can see that the values in the data can vary widely in terms of their magnitude, variability, and units.  For instance ``loan_amount`` and ``installment`` are on the order of thousands and hundreds respectively, ``term`` meanwhile has a unit of months and ``delinq_2yrs`` is a count.  For some machine learning algorithms these differences can cause training to take longer or not converge at all.  By scaling these features you can give them a similar magnitude without affecting the value's distribution.

For a good introduction to feature scaling, how it works, what it does, and when to use it see:

https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e

https://en.wikipedia.org/wiki/Feature_scaling

The functions `normalize` and `standardize` are defined here for simplicity and clarity.  Many Python frameworks such as scikit-learn provide functions out of the box for scaling and normalization.

In [ ]:
def normalize (series):
    return (series - series.min ())/(series.max () - series.min ())

def standardize (series):
    return (series - series.mean())/(series.std())

To ensure that the distribution of the feature is not affected by scaling use the ``plot`` function of the Pandas DataFrame to display the distribution before and after.

In [ ]:
%matplotlib inline

df.loan_amnt.plot.kde (title = 'loan_amnt distribution pre-scaling')

Use the `select_dtypes` function again to obtain a reference to all features of type `float64` or `int64`.  Use the `standardize` function above to reduce the magnitude of each feature.

In [ ]:
%matplotlib inline
    
df.loan_amnt.plot.kde (title = 'loan_amnt distribution post-scaling')

Note that the values of the X and Y axis have changed but the distribution of the data over the range has remained static.

In [ ]:
df.describe ()

We can see that the distribution of the field has not changed and that the variations remain the same.  However the Standard Deviation has been equalized across the fields bringing the values into a similar magnitude.

---

# TEST THE DATA

Now that the data has been cleaned and the features engineered, create a training and testing data set to quickly assess how well the engineered data performs using logistic regression.

We will use 80% of the overall data set for training and the remaining 20% of the data for testing purposes.  As logistic regression is a supervised ML algorithm we will provide it with the 'grade' feature as our label.

You will need to perform the 'one hot' encoding of the categorical data columns.  To do this use the [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) function to convert all categorical columns in the data set.

Rather than have the logistic regression algorithm predict a `one hot` encoded vector lets use the [argmax](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.argmax.html) function to reduce the `one hot` encoding to a number such as 1, 2, 3, 4, ...

In [ ]:
train_test_split_index = int (df.shape[0] * 0.8)

# Note, SKLearn does not do well with NaN values, remove any and all
df.dropna(axis=1, inplace=True)

# split the features from the labels
labels = df.pop('grade')

df = pd.get_dummies (df)
labels = pd.get_dummies (labels)
labels = labels.values.argmax (axis = 1)

train_df = df[:train_test_split_index]
train_labels = labels[:train_test_split_index]
print ("Training data has {} rows".format (train_df.shape[0]))

test_df = df[train_test_split_index:]
test_labels = labels[train_test_split_index:]
print ("Test data has {} rows".format (test_df.shape[0]))

In [ ]:
train_df.head (5)

In [ ]:
train_labels[0:5]

### Test using logistic regression

scikit-learn has many common classifiers pre-built in the library.  Lets now use the logistic regression algorithm to train a model based upon our feature data and assess how well the trained model performs on the test data set.

The logistic regression algorithm is instantiated for you, use the [fit](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.fit) function to train the algorithm based upon your training data and labels.

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression (solver = 'lbfgs')

In [ ]:

lreg.fit ( ... )


With your model trained lets now use the test data set to evaluate the performance of the model on yet unseen observations.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

print ("Accuracy for logistic regression on data set with {} features".format (train_df.shape[1]))
print (lreg.score (test_df, test_labels))

predictions = lreg.predict (test_df)
print ("Precision, recall, F1 score")
print (precision_recall_fscore_support (test_labels, predictions, average = 'macro'))

# EXTRA CREDIT: Principal Component Analysis (PCA)

When encoded our data set contains over 1000 features.  Many other data sets may contain 10s or 100s of thousands of features, some even go into the millions of features for every observation.  Unless you have a deep understanding of every field in your data set and how it relates to your target value tools like PCA can aid greatly in determineing those features that are the most influential and important for your machine learning model.

PCA is an algorithm that observes the relationship between individual features and the target label.  Using Principal Component Analysis you can reduce your feature set to be half its size, ideally without losing any model accuracy. Such a reduction in size will allow your models to train more quickly.

In addition to reducing the dimensionality of your data PCA can also ensure that all features are independent of one another.  Some machine learning algorithms will require that features be independent of one another.

Lets use the PCA class from SciKit-Learn to reduce our feature set while still being able to describe with a high degree of accuracy the target labels.

**Note:** Principal component analysis will require all numerical values and will not accept NaN.  Further engineer the data set to remove null values and convert the category data to numerical values.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA (0.99) # PCA should choose the components that account for 99% of the variance

In [ ]:
print ("The training data set holds {} records with {} features".format (train_df.shape[0], train_df.shape[1]))

Using the [fit](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.fit) function of the PCA algorithm, reduce the number of features to the optimal number needed to account for 99% of the variance in the training data.

In [ ]:
%%time
pca.fit ( ... )

Next use the [transform](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.transform) method to modify the training and test data sets to the reduced feature set.

In [ ]:
train_df = pca.transform ( ... )
test_df = pca.transform( ... )

In [ ]:
print ("After PCA analysis the data set contains {} rows with {} features".format (train_df.shape[0], train_df.shape[1]))

We can see that after analysis PCA has reduced the dimensionality of the data set to less than 1/10th the size of the original feature set.

In [ ]:
%%time
lreg = LogisticRegression (solver = 'lbfgs')
lreg.fit (X=train_df, y=train_labels)

Training a logistic regression model on the reduced data set required less time than the full feature set.  The model trained more quickly, lets see what has happened to the accuracy.

In [ ]:
print ("Accuracy for logistic regression on reduced data set with {} features".format (train_df.shape[1]))
print (lreg.score (test_df, test_labels))

predictions = lreg.predict (test_df)
print ("Precision, recall, F1 score")
print (precision_recall_fscore_support (test_labels, predictions, average = 'macro'))

Little to no impact to model accuracy and precision while cutting training time down by 66%.

## Output for training

Now that the features have been selected, cleansed, and scaled lets output them for consumption by a training algorithm.  This can be easily done using Pandas functions like [to_parquet](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_parquet.html) or [to_csv](http://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.to_csv.html).  To see what the results would look like use the ``get_dummies`` function to perform the one hot encoding of the categories to vectors.  This will give you an example of the data which is ultimately used to train your algorithm.

In [ ]:
pd.DataFrame (train_df).to_csv ('data/pca_train_features.csv', index=False)

In [ ]:
pd.DataFrame (train_labels).to_csv ('data/train_labels.csv', index=False)

In [ ]:
!ls data

In [ ]:
! head data/pca_train_features.csv

In [ ]:
! head data/train_labels.csv

## End of lab

You have now downloaded a raw data set and used tools such as Pandas to explore the data, engineer a feature set from the raw data, and optimize the data set for use with logistic regression.  The next step from here is to take the work you've developed in this Jupyter notebook and make it a part of an extraction, transformation, and loading process for reuse.  Equally the logic applied to the raw data fields will also need to be applied to future requests to the trained model.  New observartions will need to be transformed in the same manner as the training data so that the trained model can recognize it.  This can be done at the client side or within the model itself and is up to the development team.  

While logistic regression appears to be 99.9% accurate in calculating loan grade based upon the data provided what other features can you see being derived from this data set?  What are your thoughts on what data fields are the most relevant and how could they further be transformed to more accurately reflect reality?